In [1]:
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import variation
from matplotlib.ticker import MaxNLocator

from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.linear_model import LassoCV,Lasso,Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from skorch import NeuralNetRegressor

import torch
import torch.nn.functional as F 
import torch.utils.data as Data
import time
import psutil
import os
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import seaborn as sns
import json
from termcolor import colored
from tqdm import tqdm
import timeit
# import pickle
import joblib

from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score  
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler,StandardScaler,Normalizer


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression,mutual_info_regression
from statsmodels.distributions.empirical_distribution import ECDF

#Evaluation
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
warnings.filterwarnings('ignore', 'Objective did not converge.*')

#Avoiding Type 3 fonts in matplotlib plots
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
font = {'size'   : 40}

matplotlib.rc('font', **font)
matplotlib.rc('lines', linewidth=3.0)
matplotlib.rc('lines', markersize=10)

In [2]:
#read the metrics and target values from local files
# mat=pd.read_pickle('QoE_metrics_VoD(CQM_ITUT)_13952.pkl')
#if the metrics is csv file
mat=pd.read_csv('QoE_metrics_VoD(CQM_ITUT)_13952.csv',index_col=0)

#for training and testing CQM
mat=mat.rename(columns={'QoE_CQM':'QoE'})
mark='CQM'
#for training and testing ITUT
# mat=mat.rename(columns={'QoE_ITUT':'QoE'})
# mark=ITUT

mat=mat.fillna(0)
X=mat.iloc[:,-(mat.shape[1]-2):]

Y=mat['QoE']
#drop columns which have same values in all rows
# X=X.drop(X.std()[(X.std() == 0)].index, axis=1)
maxnum_feature=X.shape[1]
print(X.shape)

(13952, 96)


In [7]:
np.isinf(mat).sum().sum()

0

### Split the data into different speed of movements

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=17)

X_stationary=X_test.filter(like='_nm_', axis=0)
Y_stationary=Y_test.filter(like='_nm_', axis=0)

X_moving=X_test.drop(X_test.filter(like='_nm_', axis=0).index)
Y_moving=Y_test.drop(Y_test.filter(like='_nm_', axis=0).index)

X_pedestrians=X_moving.filter(regex='_s50-50_80_cli([0123]?[0-9]_)|_s100-0_|_s50-50_160_cli([01234567]?[0-9]_)',axis=0)
Y_pedestrians=Y_moving.filter(regex='_s50-50_80_cli([0123]?[0-9]_)|_s100-0_|_s50-50_160_cli([01234567]?[0-9]_)',axis=0)

X_vehicles=X_moving.filter(regex='_s50-50_80_cli(4[0-9]|[567][0-9]_)|_s0-100_|_s50-50_160_cli([89][0-9]|1[0-5][0-9])',axis=0)
Y_vehicles=Y_moving.filter(regex='_s50-50_80_cli(4[0-9]|[567][0-9]_)|_s0-100_|_s50-50_160_cli([89][0-9]|1[0-5][0-9])',axis=0)

X_test.shape,X_stationary.shape,X_moving.shape,X_pedestrians.shape,X_vehicles.shape


((2791, 96), (630, 96), (2161, 96), (1037, 96), (1124, 96))

### Lasso Regression
Linear Model trained with L1 prior as regularizer (aka the Lasso)

In [35]:
pipe_Lasso=Pipeline([('scale', StandardScaler()),('clf', Lasso())])
# alphas=[1e-9, 0.0001, 0.0003, 0.0005,0.00055,0.0006,0.00065,0.0007, 0.0008,0.0009,0.001, 0.003, 0.005, 0.007, 0.01, 0.03, 0.05, 0.07, 0.08, 0.09, 0.1]
alphas=[0.0001,0.0002,0.0003,0.0005,0.0007,0.001,0.005,0.01, 0.03, 0.05, 0.07, 0.08, 0.09, 0.1]
Lasso_params={'clf__alpha': alphas}
cv_Lasso = GridSearchCV(pipe_Lasso, param_grid=Lasso_params, cv=5, scoring='neg_mean_squared_error',n_jobs=-1, verbose=3,return_train_score=True)
start=time.time()
cv_Lasso.fit(X_train, Y_train)
#save the model
joblib.dump(cv_Lasso,'model_%s_Lasso_MSE.joblib'%mark)

vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()
print('the duration of training is:',finish-start)

print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   22.4s finished


Total CPUs utilized percentage : 50.7 %
the duration of training is: 23.770852088928223
Percentage of used RAM : 64.0 %


In [37]:
#Get the MSE score for different type in the test dataset by applying the same model
print('--------------Lasso------------')
print('All: ',mean_squared_error(Y_test,cv_Lasso.predict(X_test)))
print('stationary: ',mean_squared_error(Y_stationary,cv_Lasso.predict(X_stationary)))
print('moving: ',mean_squared_error(Y_moving,cv_Lasso.predict(X_moving)))
print('pedestrians: ',mean_squared_error(Y_pedestrians,cv_Lasso.predict(X_pedestrians)))
print('vehicles: ',mean_squared_error(Y_vehicles,cv_Lasso.predict(X_vehicles)))

All:  0.06698094205764775
stationary:  0.08492378969654446
moving:  0.061750033213360415
pedestrians:  0.05435027170676903
vehicles:  0.06857703737913913


In [36]:
print('Best validation score: {}'.format(cv_Lasso.best_score_))
print('Best parameters: {}'.format(cv_Lasso.best_params_))
start=time.time()
print('Test score: ',mean_squared_error(Y_test,cv_Lasso.predict(X_test)))
vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()
print('the duration of test is:',finish-start)
print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

train_scores_mean = cv_Lasso.cv_results_["mean_train_score"]
test_scores_mean = cv_Lasso.cv_results_["mean_test_score"]
print('The training score of the best estimator: ',train_scores_mean[np.argmax(test_scores_mean)])

Best validation score: -0.06912046216715706
Best parameters: {'clf__alpha': 0.0003}
Test score:  0.06698094205764775
Total CPUs utilized percentage : 4.7 %
the duration of test is: 0.00956583023071289
Percentage of used RAM : 64.2 %
The training score of the best estimator:  -0.06767183632770972


### Linear Ridge Regression 
Linear least squares with l2 regularization.

In [44]:
pipe_LRR = Pipeline([('scale', StandardScaler()),('clf', Ridge())])

# alphas=[1e-9,1e-8,1e-7,1e-6,1e-5,1e-4, 0.0003, 0.0005, 0.0007, 0.001, 0.003, 0.005, 0.007, 0.01, 0.03, 0.05, 0.07, 0.08, 0.09, 0.1, 0.3, 0.5, 0.7,1,5,7,10,100,500] 
alphas=[1e-4, 0.0003, 0.0005, 0.0007,0.001, 0.003, 0.005, 0.007,0.01,0.1,70] 

LRR_params ={'clf__alpha': alphas}

cv_LRR = GridSearchCV(pipe_LRR, param_grid=LRR_params, cv=5, scoring='neg_mean_squared_error',n_jobs=-1, verbose=3,return_train_score=True)
start=time.time()
cv_LRR.fit(X_train, Y_train)
#save the model
joblib.dump(cv_LRR,'model_%s_LRR_MSE.joblib'%mark)

vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()
print('the duration of training is:',finish-start)

print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.0s


Total CPUs utilized percentage : 36.9 %
the duration of training is: 4.0333569049835205
Percentage of used RAM : 66.7 %
CPU times: user 1.01 s, sys: 93.8 ms, total: 1.11 s
Wall time: 4.05 s


[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    3.8s finished


In [46]:
#Get the MSE score for different type in the test dataset by applying the same model
print('--------------LRR------------')
print('All: ',mean_squared_error(Y_test,cv_LRR.predict(X_test)))
print('stationary: ',mean_squared_error(Y_stationary,cv_LRR.predict(X_stationary)))
print('moving: ',mean_squared_error(Y_moving,cv_LRR.predict(X_moving)))
print('pedestrians: ',mean_squared_error(Y_pedestrians,cv_LRR.predict(X_pedestrians)))
print('vehicles: ',mean_squared_error(Y_vehicles,cv_LRR.predict(X_vehicles)))

All:  0.06312404837660893
stationary:  0.0789270533337478
moving:  0.058516971503403244
pedestrians:  0.05161448534543975
vehicles:  0.06488519049433575


In [45]:
print('Best validation score: {}'.format(cv_LRR.best_score_))
print('Best parameters: {}'.format(cv_LRR.best_params_))
start=time.time()
print('Test score: ',mean_squared_error(Y_test,cv_LRR.predict(X_test)))
vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()
print('the duration of test is:',finish-start)
print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

train_scores_mean = cv_LRR.cv_results_["mean_train_score"]
test_scores_mean = cv_LRR.cv_results_["mean_test_score"]
print('The training score of the best estimator: ',train_scores_mean[np.argmax(test_scores_mean)])


Best validation score: -0.06558946784201304
Best parameters: {'clf__alpha': 0.0001}
Test score:  0.06312404837660893
Total CPUs utilized percentage : 24.7 %
the duration of test is: 0.007996320724487305
Percentage of used RAM : 66.3 %
The training score of the best estimator:  -0.06378445681296452


## Kernel Ridge Regression

#### RBF Kernel

In [52]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', KernelRidge())
])

param_grid = [
    {
        'clf__kernel': ['rbf'],
#         'clf__alpha': [0, 0.0001, 0.0003, 0.0005, 0.0007, 0.001, 0.003, 0.005, 0.007, 0.01, 0.03, 0.05, 0.07, 0.08, 0.09, 0.1, 0.3, 0.5, 0.7, 1, 3, 5, 10]
        'clf__alpha': [0.01, 0.03, 0.05, 0.07,0.1, 0.2,0.5, 0.7, 1,10]

    },
]


#scoring=neg_mean_squared_error
cv_krr = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error',n_jobs=-1, verbose=3,return_train_score=True)
start=time.time()
cv_krr.fit(X_train, Y_train)
#save the model
joblib.dump(cv_krr,'model_%s_krr_MSE.joblib'%mark)

vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()
print('the duration of test is:',finish-start)
print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  7.6min finished


Total CPUs utilized percentage : 92.8 %
the duration of test is: 472.01826095581055
Percentage of used RAM : 28.7 %
CPU times: user 21 s, sys: 1.75 s, total: 22.8 s
Wall time: 7min 52s


In [54]:
#Get the MSE score for different type in the test dataset by applying the same model
print('--------------KRR------------')
print('All: ',mean_squared_error(Y_test,cv_krr.predict(X_test)))
print('stationary: ',mean_squared_error(Y_stationary,cv_krr.predict(X_stationary)))
print('moving: ',mean_squared_error(Y_moving,cv_krr.predict(X_moving)))
print('pedestrians: ',mean_squared_error(Y_pedestrians,cv_krr.predict(X_pedestrians)))
print('vehicles: ',mean_squared_error(Y_vehicles,cv_krr.predict(X_vehicles)))

All:  0.05765112427975837
stationary:  0.07485071559042569
moving:  0.05263689821510291
pedestrians:  0.04486412397369146
vehicles:  0.05980804313355823


In [53]:
print('Best validation score: {}'.format(cv_krr.best_score_))
print('Best parameters: {}'.format(cv_krr.best_params_))
start=time.time()
print('Test score: ',mean_squared_error(Y_test,cv_krr.predict(X_test)))
vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()
print('the duration of test is:',finish-start)
print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

train_scores_mean = cv_krr.cv_results_["mean_train_score"]
test_scores_mean = cv_krr.cv_results_["mean_test_score"]
print('The training score of the best estimator: ',train_scores_mean[np.argmax(test_scores_mean)])

Best validation score: -0.06443499825540126
Best parameters: {'clf__alpha': 0.07, 'clf__kernel': 'rbf'}
Test score:  0.05765112427975837
Total CPUs utilized percentage : 5.8 %
the duration of test is: 0.7514591217041016
Percentage of used RAM : 38.1 %
The training score of the best estimator:  -0.0339930702108747


## Support Vector Regression

In [61]:
pipe_SVR = Pipeline([('scale', StandardScaler()),('clf', SVR())])
SVR_params = [
    {
        'clf__C': np.linspace(0.1,5, num=20),
        'clf__gamma': [0.003,0.04,0.005,0.06,0.007,0.01],
        'clf__kernel': ['rbf']
    },
]
cv_SVR = GridSearchCV(pipe_SVR, param_grid=SVR_params, cv=5, scoring='neg_mean_squared_error',n_jobs=-1, verbose=3,return_train_score=True)
start=time.time()
cv_SVR.fit(X_train, Y_train)
#save the model
joblib.dump(cv_SVR,'model_%s_SVR_MSE.joblib'%mark)

vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()
print('the duration of training is:',finish-start)
print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed: 41.0min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 48.7min finished


Total CPUs utilized percentage : 95.4 %
the duration of training is: 2933.6711165905
Percentage of used RAM : 58.7 %
CPU times: user 19.7 s, sys: 930 ms, total: 20.7 s
Wall time: 48min 53s


In [63]:
#Get the MSE score for different type in the test dataset by applying the same model
print('--------------SVR------------')
print('All: ',mean_squared_error(Y_test,cv_SVR.predict(X_test)))
print('stationary: ',mean_squared_error(Y_stationary,cv_SVR.predict(X_stationary)))
print('moving: ',mean_squared_error(Y_moving,cv_SVR.predict(X_moving)))
print('pedestrians: ',mean_squared_error(Y_pedestrians,cv_SVR.predict(X_pedestrians)))
print('vehicles: ',mean_squared_error(Y_vehicles,cv_SVR.predict(X_vehicles)))

All:  0.05477098297547724
stationary:  0.06335801464864224
moving:  0.05226759104854807
pedestrians:  0.04390564385601982
vehicles:  0.05998230567368311


In [62]:
print('Best validation score: {}'.format(cv_SVR.best_score_))
print('Best parameters: {}'.format(cv_SVR.best_params_))
start=time.time()
print('Test score: ',mean_squared_error(Y_test,cv_SVR.predict(X_test)))
vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()
print('the duration of test is:',finish-start)
print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

train_scores_mean = cv_SVR.cv_results_["mean_train_score"]
test_scores_mean = cv_SVR.cv_results_["mean_test_score"]
print('The training score of the best estimator: ',train_scores_mean[np.argmax(test_scores_mean)])

Best validation score: -0.05545148822753737
Best parameters: {'clf__C': 5.0, 'clf__gamma': 0.005, 'clf__kernel': 'rbf'}
Test score:  0.05477098297547724
Total CPUs utilized percentage : 4.0 %
the duration of test is: 1.5488080978393555
Percentage of used RAM : 59.1 %
The training score of the best estimator:  -0.044610899839142806


## Neural Network

In [ ]:
#split the dataset
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.2,random_state=17)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.25, random_state=7)

X_stationary=xtest.filter(like='_nm_', axis=0)
Y_stationary=ytest.filter(like='_nm_', axis=0)

X_moving=xtest.drop(xtest.filter(like='_nm_', axis=0).index)
Y_moving=ytest.drop(ytest.filter(like='_nm_', axis=0).index)

X_pedestrians=X_moving.filter(regex='_s50-50_80_cli([0123]?[0-9]_)|_s100-0_|_s50-50_160_cli([01234567]?[0-9]_)',axis=0)
Y_pedestrians=Y_moving.filter(regex='_s50-50_80_cli([0123]?[0-9]_)|_s100-0_|_s50-50_160_cli([01234567]?[0-9]_)',axis=0)

X_vehicles=X_moving.filter(regex='_s50-50_80_cli(4[0-9]|[567][0-9]_)|_s0-100_|_s50-50_160_cli([89][0-9]|1[0-5][0-9])',axis=0)
Y_vehicles=Y_moving.filter(regex='_s50-50_80_cli(4[0-9]|[567][0-9]_)|_s0-100_|_s50-50_160_cli([89][0-9]|1[0-5][0-9])',axis=0)


#convert dataframe to tensor
xtrain=torch.Tensor(xtrain.values)
xtest=torch.Tensor(xtest.values)
ytrain=torch.Tensor(ytrain.values).reshape(-1,1)
ytest=torch.Tensor(ytest.values).reshape(-1,1)

xval=torch.Tensor(xval.values)
yval=torch.Tensor(yval.values).reshape(-1,1)

X_stationary=torch.Tensor(X_stationary.values)
Y_stationary=torch.Tensor(Y_stationary.values).reshape(-1,1)
X_moving=torch.Tensor(X_moving.values)
Y_moving=torch.Tensor(Y_moving.values).reshape(-1,1)
X_pedestrians=torch.Tensor(X_pedestrians.values)
Y_pedestrians=torch.Tensor(Y_pedestrians.values).reshape(-1,1)
X_vehicles=torch.Tensor(X_vehicles.values)
Y_vehicles=torch.Tensor(Y_vehicles.values).reshape(-1,1)

print(xtest.shape,X_stationary.shape,X_moving.shape,X_pedestrians.shape,X_vehicles.shape)
print(xtrain.shape, xtest.shape,xval.shape)

In [ ]:
class RegressorNet(torch.nn.Module):
    def __init__(self, n_feature=10, n_hidden=10, n_output=1):
        super(RegressorNet, self).__init__()
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden)  # 95-(100-10)-1
        self.predict = torch.nn.Linear(n_hidden, n_output)
    
    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = self.predict(x)  # no activation, aka Identity()
        return x

In [ ]:
# Hyper Parameters
EPOCH = 1500               # train the training data 500 times
BATCH_SIZE = 300
LR = 0.002

torch_dataset=Data.TensorDataset(xtrain,ytrain)
trainloader=Data.DataLoader(dataset=torch_dataset,batch_size=BATCH_SIZE,shuffle=True)

start=time.time()
net=RegressorNet(n_feature=maxnum_feature,n_hidden=50,n_output=1)
opt=torch.optim.Adam(net.parameters(),lr=LR)
loss_func=torch.nn.MSELoss()
meanloss_train=[]
meanloss_valid=[]

for epoch in range(EPOCH):
#     print('Epoch:',epoch)
    step_loss=[]
    for step,(batchx,batchy) in enumerate(trainloader):
        output=net(batchx)
        loss=loss_func(output,batchy)
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        step_loss.append(loss.data.numpy().flatten()[0])
#     print(step_loss)

    meanloss_train.append(np.array(step_loss).mean())
    valpre=net(xval)
    val_loss=loss_func(valpre,yval)
    meanloss_valid.append(val_loss.data.numpy().flatten()[0])
    
vcpu=psutil.cpu_percent()
print ('Total CPUs utilized percentage :',vcpu,'%')
finish=time.time()  
print('the duration of training is:',finish-start)
print('Percentage of used RAM :',psutil.virtual_memory().percent,'%')

In [ ]:
#Get the MSE score for different type in the test dataset by applying the same model
print('----------------NN---------------')
print('All: ',loss_func(net(xtest),ytest).data.numpy())
print('stationary: ',loss_func(net(X_stationary),Y_stationary).data.numpy())
print('moving: ',loss_func(net(X_moving),Y_moving).data.numpy())
print('pedestrians: ',loss_func(net(X_pedestrians),Y_pedestrians).data.numpy())
print('vehicles: ',loss_func(net(X_vehicles),Y_vehicles).data.numpy())

In [ ]:
#save the model
torch.save(net.state_dict(), 'model_%s_NN_MSE.pt'%mark)